In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df_go = pd.read_csv("./go_enrich.tsv", sep="\t")
df_go

,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count,category
0,GO:0003924,GTPase activity,24/646,71/6066,1.231164e-07,1.292722e-05,1.088608e-05,ANI_1_1776014/ANI_1_352154/ANI_1_124/ANI_1_250...,24,MF
1,GO:0016887,ATPase activity,31/646,111/6066,2.652500e-07,1.392562e-05,1.172684e-05,ANI_1_1902094/ANI_1_2222094/ANI_1_1288084/ANI_...,31,MF
2,GO:0005525,GTP binding,26/646,88/6066,7.525797e-07,2.634029e-05,2.218130e-05,ANI_1_1776014/ANI_1_352154/ANI_1_124/ANI_1_250...,26,MF
3,GO:0003676,nucleic acid binding,45/646,231/6066,3.451899e-05,9.061235e-04,7.630514e-04,ANI_1_430184/ANI_1_2142064/ANI_1_2152074/ANI_1...,45,MF
4,GO:0004842,ubiquitin-protein transferase activity,9/646,19/6066,5.753337e-05,1.208201e-03,1.017432e-03,ANI_1_212054/ANI_1_168134/ANI_1_386014/ANI_1_2...,9,MF
5,GO:0005085,guanyl-nucleotide exchange factor activity,10/646,24/6066,8.547509e-05,1.495814e-03,1.259633e-03,ANI_1_304164/ANI_1_898134/ANI_1_450134/ANI_1_4...,10,MF
6,GO:0017056,structural constituent of nuclear pore,6/646,10/6066,2.055110e-04,3.082665e-03,2.595929e-03,ANI_1_1918184/ANI_1_1460094/ANI_1_20124/ANI_1_...,6,MF
7,GO:0004672,protein kinase activity,30/646,150/6066,4.385185e-04,5.755555e-03,4.846783e-03,ANI_1_1450164/ANI_1_518154/ANI_1_2808024/ANI_1...,30,MF
8,GO:0003723,RNA binding,32/646,167/6066,6.308164e-04,7.359525e-03,6.197495e-03,ANI_1_170054/ANI_1_1646014/ANI_1_1738024/ANI_1...,32,MF
9,GO:0031267,small GTPase binding,6/646,14/6066,2.019589e-03,2.120568e-02,1.785741e-02,ANI_1_386064/ANI_1_776164/ANI_1_1664134/ANI_1_...,6,MF


In [4]:
import plotly.express as px

def GO_BarChart(df_go, color='Geyser', go_type='all', p_value=0.5, bar_num=30, width=1000, height=800):
    """根据输入的GO富集分析结果，绘制柱状图
    """

    # 数据处理
    df_go = df_go.copy()
    df_go = df_go[["category", "ID", "Description", "Count", 'GeneRatio', "p.adjust"]]
    df_go.columns = ["Class", "ID", "Description", "Count", "GeneRatio", "P.adjust"]

    df_go["GeneRatio"] = df_go["GeneRatio"].apply(lambda x: round(eval(x), 3))
    df_go['P.adjust'] = df_go['P.adjust'].apply(lambda x: round(x, 6))
    df_go = df_go.sort_values(by='Count', ascending=False)
    df_go = df_go[df_go["P.adjust"] < p_value]
    df_go = df_go.iloc[:bar_num]

    # 过滤GO类型
    if go_type in ["BP", "CC", "MF"]:
        df_go = df_go[df_go["Class"].str.contains(go_type)]

    # 创建柱状图
    fig = px.bar(
        df_go, 
        x='Count',
        y='Description',
        color='P.adjust',
        color_continuous_scale=color,
        opacity=0.85,
        hover_data=["Class", "ID", "GeneRatio", "P.adjust"],
        )
    fig.update_layout(
        title="GO Enrichment Analysis",
        xaxis_title="Count",
        yaxis_title="Description",
        yaxis=dict(autorange="reversed"),
        font=dict(family="Arial", size=14),
        template="plotly_white",
        width=width,
        height=height,
    )
    fig.update_coloraxes(
        colorbar_title="P.adjust",
        colorbar_tickformat=".3f",
        colorbar=dict(dtick=0.005)
    )

    return fig

# 调用函数示例
GO_BarChart(df_go, go_type="")
